## Convert .mat file to netcdf for Mooring D Data 

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy.io import loadmat
from datetime import datetime as dt
import glob
from tqdm import tqdm

In [2]:
# Carson's Function to read MORSea matlab files
def read_MORSea_mat(filepath):
    '''
    Opens matlab file from MORSea that is assumed to contain 2 structs named 'dat' and 'hdr'.
    'hdr' must contain variables 'start_time','stop_time',and 'samp_int_minutes', from which the timebase will be constructed. If the resulting datetime array is longer than the data arrays, it is assumed that the start time is correct.
    An xarray dataset is created with all variables in 'dat' (assumes there are at least 2). Fields in 'hdr' are stored as metadata. 
    C. Witte 10/2025
    '''

    #open file and extract variable names
    mat = loadmat(filepath)
    data_fields = mat['dat'].dtype.names
    hdr_fields = mat['hdr'].dtype.names

    #construct timebase
    start = dt.strptime(mat['hdr']['start_time'][0][0][0].lstrip('0'), '%d/%m/%Y %H:%M:%S')
    stop = dt.strptime(mat['hdr']['stop_time'][0][0][0].lstrip('0'), '%d/%m/%Y %H:%M:%S') 
    interval = str(mat['hdr']['samp_int_minutes'][0][0][0][0])+'min'
    datetime_array = pd.date_range(start=start, end=stop, freq=interval)[:len(mat['dat'][data_fields[0]][0][0])] #force the datetime array to be the same length as the data
    diff = len(mat['dat'][data_fields[0]][0][0]) - len(datetime_array) #check if the datetime array is too short relative to the data and fix if needed
    if diff>0:
        datetime_array = pd.date_range(start=start, freq=interval, periods=len(datetime_array)+diff)
        
    #construct data_variables dictionary
    data_variables = {data_fields[0]:(('time','depth'),mat['dat'][data_fields[0]][0][0])}
    for idx in np.arange(1,len(data_fields)):
        name = data_fields[idx]
        data_variables[name] = (('time','depth'),mat['dat'][name][0][0])

    #construct attributes dictionary
    attrs = {hdr_fields[0]:mat['hdr'][hdr_fields[0]][0][0].item()}
    for idx in np.arange(1,len(hdr_fields)):
        name = hdr_fields[idx]
        attrs[name] = mat['hdr'][name][0][0].item()

    #construct xarray dataset
    out = xr.Dataset(data_vars=data_variables,
                     coords={'depth':[attrs['instr_depth_meters']],
                             'time':datetime_array},
                     attrs=attrs)
    
    return out

In [3]:
folders = glob.glob('//thepenguin/penguin2/Data/TNB/mooring/Mooring_D_MORSea/Mooring_D_2014_2025/Mooring_D/*')
for folder in tqdm(folders):
    files = glob.glob(folder+'/*.mat')
    for file in files:
        ds = read_MORSea_mat(file)
        name = file.split('\\')[-1].split('.mat')[0] #extract filename to use for netcdf save
        ds.to_netcdf(folder+'/'+name+'.nc') 

 10%|█         | 1/10 [00:00<00:02,  3.35it/s]


ValueError: Variable 'DateTime': Could not convert tuple of form (dims, data[, attrs, encoding]): (('time', 'depth'), MatlabOpaque([(b'', b'MCOS', b'datetime', array([[3707764736],
                     [         2],
                     [         1],
                     [         1],
                     [         1],
                     [         1]], dtype=uint32))            ],
             dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')])) to Variable.

Carson's function does not seem to automatically work for other Mooring D data folder, so I will try to follow his steps piecewise with one file to understand how everything works. I will be looking at the folder 'D_2017_2019'

In [136]:
#open file and extract variable names
filepath = '//thepenguin/penguin2/Data/TNB/mooring/Mooring_D_MORSea/Mooring_D_2014_2025/Mooring_D/D_2017_2019/D_2017_RCM7_11199_576_m.mat'
mat = loadmat(filepath)
data_fields = mat['dat'].dtype.names
hdr_fields = mat['hdr'].dtype.names

In [137]:
data_fields

('Mod_0', 'Dir_0', 'DateTime', 'U_0', 'V_0')

In [138]:
hdr_fields

('cruise_deployment',
 'cruise_recovery',
 'water_depth_meters',
 'instr_depth_meters',
 'instr_type',
 'instr_sn',
 'latitude_Decimal_Degrees_North',
 'longitude_Decimal_Degrees_East',
 'samp_int_minutes',
 'start_time',
 'stop_time',
 'level_0',
 'level_1',
 'level_2')

In [139]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Nov 24 22:39:42 2024',
 '__version__': '1.0',
 '__globals__': [],
 'dat': array([[(array([[36.26],
                [10.69],
                [ 5.46],
                ...,
                [ 2.55],
                [ 4.88],
                [ 5.75]], shape=(7704, 1)), array([[349.2 ],
                [ 59.55],
                [ 95.25],
                ...,
                [ 65.5 ],
                [ 76.7 ],
                [ 80.9 ]], shape=(7704, 1)), MatlabOpaque([(b'', b'MCOS', b'datetime', array([[3707764736],
                              [         2],
                              [         1],
                              [         1],
                              [         1],
                              [         1]], dtype=uint32))            ],
                      dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')]), array([[-6.79444647],
                [ 9.21554693],
                [ 5.437094

In [140]:
#construct timebase
start = dt.strptime(mat['hdr']['start_time'][0][0][0].lstrip('0'), '%d/%m/%Y %H:%M:%S')
stop = dt.strptime(mat['hdr']['stop_time'][0][0][0].lstrip('0'), '%d/%m/%Y %H:%M:%S') 
interval = str(mat['hdr']['samp_int_minutes'][0][0][0][0])+'min'
datetime_array = pd.date_range(start=start, end=stop, freq=interval)[:len(mat['dat'][data_fields[0]][0][0])] #force the datetime array to be the same length as the data
diff = len(mat['dat'][data_fields[0]][0][0]) - len(datetime_array) #check if the datetime array is too short relative to the data and fix if needed
if diff>0:
    datetime_array = pd.date_range(start=start, freq=interval, periods=len(datetime_array)+diff)

In [143]:
start

datetime.datetime(2017, 2, 11, 7, 0)

In [144]:
interval

'30min'

In [158]:
## Note that this is only for RCM instrument - the wrong interval was used above, should be 60 min, not 30 min. 
## Verified by looking at DateTime variable in original matlab file.
# interval = '60min'

In [146]:
stop

datetime.datetime(2019, 2, 18, 22, 0)

In [147]:
datetime_array = pd.date_range(start=start, end=stop, freq=interval)[:len(mat['dat'][data_fields[0]][0][0])] #force the datetime array to be the same length as the data
diff = len(mat['dat'][data_fields[0]][0][0]) - len(datetime_array) #check if the datetime array is too short relative to the data and fix if needed
if diff>0:
    datetime_array = pd.date_range(start=start, freq=interval, periods=len(datetime_array)+diff)

In [148]:
datetime_array

DatetimeIndex(['2017-02-11 07:00:00', '2017-02-11 08:00:00',
               '2017-02-11 09:00:00', '2017-02-11 10:00:00',
               '2017-02-11 11:00:00', '2017-02-11 12:00:00',
               '2017-02-11 13:00:00', '2017-02-11 14:00:00',
               '2017-02-11 15:00:00', '2017-02-11 16:00:00',
               ...
               '2017-12-28 21:00:00', '2017-12-28 22:00:00',
               '2017-12-28 23:00:00', '2017-12-29 00:00:00',
               '2017-12-29 01:00:00', '2017-12-29 02:00:00',
               '2017-12-29 03:00:00', '2017-12-29 04:00:00',
               '2017-12-29 05:00:00', '2017-12-29 06:00:00'],
              dtype='datetime64[ns]', length=7704, freq='60min')

In [149]:
#construct data_variables dictionary
data_variables = {data_fields[0]:(('time','depth'),mat['dat'][data_fields[0]][0][0])}
for idx in np.arange(1,len(data_fields)):
    name = data_fields[idx]
    data_variables[name] = (('time','depth'),mat['dat'][name][0][0])

In [150]:
# remove DateTime variables 
data_variables.pop('DateTime')

(('time', 'depth'),
 MatlabOpaque([(b'', b'MCOS', b'datetime', array([[3707764736],
                      [         2],
                      [         1],
                      [         1],
                      [         1],
                      [         1]], dtype=uint32))            ],
              dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')]))

In [151]:
data_variables

{'Mod_0': (('time', 'depth'),
  array([[36.26],
         [10.69],
         [ 5.46],
         ...,
         [ 2.55],
         [ 4.88],
         [ 5.75]], shape=(7704, 1))),
 'Dir_0': (('time', 'depth'),
  array([[349.2 ],
         [ 59.55],
         [ 95.25],
         ...,
         [ 65.5 ],
         [ 76.7 ],
         [ 80.9 ]], shape=(7704, 1))),
 'U_0': (('time', 'depth'),
  array([[-6.79444647],
         [ 9.21554693],
         [ 5.4370949 ],
         ...,
         [ 2.32040124],
         [ 4.7491129 ],
         [ 5.67762939]], shape=(7704, 1))),
 'V_0': (('time', 'depth'),
  array([[35.61773571],
         [ 5.41754508],
         [-0.49959884],
         ...,
         [ 1.05746777],
         [ 1.12264272],
         [ 0.90940889]], shape=(7704, 1)))}

In [152]:
#construct attributes dictionary
attrs = {hdr_fields[0]:mat['hdr'][hdr_fields[0]][0][0].item()}
for idx in np.arange(1,len(hdr_fields)):
    name = hdr_fields[idx]
    attrs[name] = mat['hdr'][name][0][0].item()

In [153]:
attrs

{'cruise_deployment': 'PNRA - XXXII Expedition - 2016/17  ',
 'cruise_recovery': 'PNRA - XXXIV Expedition - 2019/20',
 'water_depth_meters': -1144,
 'instr_depth_meters': '-576',
 'instr_type': 'Aanderaa RCM7',
 'instr_sn': 11199,
 'latitude_Decimal_Degrees_North': -75.13611,
 'longitude_Decimal_Degrees_East': 164.58958,
 'samp_int_minutes': 30,
 'start_time': '11/02/2017 07:00:00',
 'stop_time': '18/02/2019 22:00:00',
 'level_0': 'Raw data',
 'level_1': 'Despiking',
 'level_2': 'CTD quality check'}

In [154]:
#construct xarray dataset
out = xr.Dataset(data_vars=data_variables,
                    coords={'depth':[attrs['instr_depth_meters']],
                            'time':datetime_array},
                    attrs=attrs)

In [155]:
out

<xarray.Dataset> Size: 308kB
Dimensions:  (time: 7704, depth: 1)
Coordinates:
  * depth    (depth) <U4 16B '-576'
  * time     (time) datetime64[ns] 62kB 2017-02-11T07:00:00 ... 2017-12-29T06...
Data variables:
    Mod_0    (time, depth) float64 62kB 36.26 10.69 5.46 6.62 ... 2.55 4.88 5.75
    Dir_0    (time, depth) float64 62kB 349.2 59.55 95.25 ... 65.5 76.7 80.9
    U_0      (time, depth) float64 62kB -6.794 9.216 5.437 ... 2.32 4.749 5.678
    V_0      (time, depth) float64 62kB 35.62 5.418 -0.4996 ... 1.123 0.9094
Attributes: (12/14)
    cruise_deployment:               PNRA - XXXII Expedition - 2016/17  
    cruise_recovery:                 PNRA - XXXIV Expedition - 2019/20
    water_depth_meters:              -1144
    instr_depth_meters:              -576
    instr_type:                      Aanderaa RCM7
    instr_sn:                        11199
    ...                              ...
    samp_int_minutes:                30
    start_time:                      11/02/2017 07:00:00
    stop_time:                       18/02/2019 22:00:00
    level_0:                         Raw data
    level_1:                         Despiking
    level_2:                         CTD quality check

In [125]:
D_2017_seaguard_1847_911_m = out 

In [110]:
D_2017_seaguard_444_1135_m = out 

In [95]:
D_2017_SBE39_30492_996_m = out 

In [80]:
D_2017_SBE37_15451_772_m = out

In [59]:
D_2017_SBE37_15450_1086_m = out

In [43]:
D_2017_SBE16_1433_556_m = out 

In [156]:
D_2017_RCM7_11199_576_m = out

In [ ]:
# D_2017_RCM7_11199_576_m.to_netcdf('//thepenguin/penguin2/Data/TNB/mooring/Mooring_D_MORSea/Mooring_D_2014_2025/Mooring_D/D_2017_2019/D_2017_RCM7_11199_576_m.nc')

In [126]:
# save all datasets to netcdf 
names = ['D_2017_RCM7_11199_576_m',
         'D_2017_SBE37_15450_1086_m',
         'D_2017_SBE16_1433_556_m',
         'D_2017_SBE37_15451_772_m',
         'D_2017_SBE39_30492_996_m',
         'D_2017_seaguard_444_1135_m',
         'D_2017_seaguard_1847_911_m']
for name in names:
    ds = eval(name)
    ds.to_netcdf('//thepenguin/penguin2/Data/TNB/mooring/Mooring_D_MORSea/Mooring_D_2014_2025/Mooring_D/D_2017_2019/'+name+'.nc')

In [135]:
D_2017_seaguard_444_1135_m

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 35357, depth: 1)
Coordinates:
  * depth    (depth) <U5 20B '-1135'
  * time     (time) datetime64[ns] 283kB 2017-02-11T07:02:19 ... 2019-02-17T2...
Data variables:
    U_0      (time, depth) float64 283kB 5.177 6.875 6.247 ... 13.32 11.37 14.38
    V_0      (time, depth) float64 283kB 4.035 3.794 3.457 ... 7.255 5.705 7.978
    Mod_0    (time, depth) float64 283kB 6.564 7.852 7.14 ... 15.16 12.72 16.45
    Dir_0    (time, depth) float64 283kB 52.07 61.11 61.04 ... 61.42 63.35 60.99
    T_0      (time, depth) float64 283kB -1.89 -1.89 -1.89 ... -1.896 -1.897
    TiltX    (time, depth) float64 283kB -0.188 -0.185 -0.174 ... -0.164 -0.153
    TiltY    (time, depth) float64 283kB -0.604 -0.563 -0.592 ... -1.847 -1.91
    Heading  (time, depth) float64 283kB 24.2 24.84 25.65 ... 20.16 19.35 19.5
Attributes: (12/14)
    cruise_deployment:               PNRA - XXXII Expedition - 2016/17  
    cruise_recovery:                 PNRA - XXXIV Expedition - 2019/20
    water_depth_meters:              -1144
    instr_depth_meters:              -1135
    instr_type:                      Aanderaa Seaguard
    instr_sn:                        444
    ...                              ...
    samp_int_minutes:                30
    start_time:                      11/02/2017 07:02:19
    stop_time:                       17/02/2019 21:02:19
    level_0:                         Raw data
    level_1:                         Despiking
    level_2:                         CTD quality check